### Imporinting required Libraries

In [ ]:
import numpy as np
from scipy.spatial import distance as dist
import numpy as np
import argparse
import time
import dlib
import cv2
import os
import pandas as pd
import statistics
from statistics import mean
import math
import pickle

### Importing 68 face detection landmark


In [ ]:
import urllib.request
urllib.request.urlretrieve('http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2', 'shape_predictor_68_face_landmarks.dat.bz2')

#Uncompress file
import bz2,shutil

with bz2.BZ2File("shape_predictor_68_face_landmarks.dat.bz2") as fr, open("shape_predictor_68_face_landmarks.dat","wb") as fw:
    shutil.copyfileobj(fr,fw)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
dlib_detector = dlib.get_frontal_face_detector()
dlib_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

EAR_THRESHOLD = 0.21 
EAR_CONSEC_FRAMES = 3 
SKIP_FIRST_FRAMES = 150

## Feature Extraction


---

The feature like Eye Aspect Ratio
(EAR), Mouth aspect ratio (MAR), Eye Circularity (EC),
PERCLOS, Level of Eyebrows (LEB) and Size of Pupil (SOP) is extracted from each frames using below functions. 


In [ ]:
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [ ]:
def mouth_aspect_ration(mouth):
    A = dist.euclidean(mouth[0], mouth[1])
    B = dist.euclidean(mouth[2], mouth[3])
    mar = A / B
    return mar

In [ ]:
def eye_circularity(eye):

    A = dist.euclidean(eye[0], eye[1])
    B = dist.euclidean(eye[1], eye[2])
    C = dist.euclidean(eye[2], eye[3])
    D = dist.euclidean(eye[3], eye[4])
    E = dist.euclidean(eye[4], eye[5])
    F = dist.euclidean(eye[5], eye[0])
    eye_perimeter = A + B + C + D + E + F

    radius = dist.euclidean(eye[1], eye[4]) / 2 

    pupil_area = (radius * radius)  * math.pi

    eye_circ = (4 * pupil_area * math.pi) / (eye_perimeter * eye_perimeter)

    return eye_circ

In [ ]:
def compute_perclos(perclosList):

    avg = mean(perclosList)
    perclos_percentage = avg * 100

    return perclos_percentage

In [ ]:
def level_of_eyebrows(eye):

    A = dist.euclidean(eye[0], eye[2])
    B = dist.euclidean(eye[1], eye[2])

    leb = (A + B) / 2
    return  leb

In [ ]:
def size_of_pupil(eye):
    A = dist.euclidean(eye[1], eye[4])
    B = dist.euclidean(eye[0], eye[3])

    sop = A / B
    return sop

Using this `process_video` function we extract each frames the video and run the feature extraction functions on each frames.

In [ ]:
def process_video(input_file, subject, external_factors, facial_actions, detector=dlib_detector, predictor=dlib_predictor, \
                  ear_th=0.21, consec_th=3, skip_n=150, up_to=None):
    # define necessary variables
    COUNTER = 0
    TOTAL = 0
    current_frame = 1
    blink_start = 1
    blink_end = 1
    closeness = 0
    frame_info_list = []
    video_info_list = []

    perclos_list = []
    perclos = 0

    # define capturing method
    vcap = cv2.VideoCapture(input_file)
    time.sleep(1.0)

    # build a dictionary video_info
    fps = vcap.get(cv2.CAP_PROP_FPS)
    frame_count = int(vcap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps
    video_info_dict = {
        'fps': fps,
        'frame_count': frame_count,
        'duration(s)': duration,
    }

    while True:
        (grabbed, frame) = vcap.read()
        if not grabbed:
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # detect faces in the grayscale frame
        rects = detector(gray, 0)
        
        if len(rects) == 0:
            frame_info = {
                'subject': subject,
                'external_factors': external_factors,
                'facial_actions': facial_actions,
                'frame_no': current_frame,
                'face_detected': 0,
                'left_ear': -1,
                'right_ear': -1,
                'avg_ear': -1,
                'mar': -1,
                'moe': -1,
                'left_ec' : -1,
                'right_ec' : -1,
                'avg_ec' : -1,
                'left_leb': -1,
                'right_leb': -1,
                'avg_leb': -1,
                'left_sop': -1,
                'right_sop': -1,
                'avg_sop': -1,
                'closeness': -1,
                'blink_no': -1,
                'blink_start_frame': -1,
                'blink_end_frame': -1,
                'reserved_for_calibration': False,
                'perclos': -1,
                }
            
        if len(rects) > 1:
            frame_info = {
                'subject': subject,
                'external_factors': external_factors,
                'facial_actions': facial_actions,
                'frame_no': current_frame,
                'face_detected': len(rects),
                'left_ear': -1,
                'right_ear': -1,
                'avg_ear': -1,
                'mar': -1,
                'moe': -1,
                'left_ec' : -1,
                'right_ec' : -1,
                'avg_ec' : -1,
                'left_leb': -1,
                'right_leb': -1,
                'avg_leb': -1,
                'left_sop': -1,
                'right_sop': -1,
                'avg_sop': -1,
                'closeness': -1,
                'blink_no': -1,
                'blink_start_frame': -1,
                'blink_end_frame': -1,
                'reserved_for_calibration': False,
                'perclos': -1,
                }
            
        # loop over the face detections
        if len(rects) == 1:
            rect=rects[0]
            shape = predictor(gray, rect)
            shape = np.array([[p.x, p.y] for p in shape.parts()])
            

            # extract the left and right eye coordinates, then use the
            # coordinates to compute the eye aspect ratio for both eyes
            leftEye = shape[42:48]
            rightEye = shape[36:42]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            # average the eye aspect ratio together for both eyes
            ear = (leftEAR + rightEAR) / 2.0

             # mouth aspect ratio
            mouth = [shape[62], shape[66], shape[60], shape[64]]
            mar = mouth_aspect_ration(mouth)

            #mouth over eye
            moe = mar/ear

            #eye circularity
            leftEC = eye_circularity(leftEye)
            rightEC = eye_circularity(rightEye)

            eye_circ = (leftEC + rightEC) / 2.0

            #level of eyebrows
            right_eye_leb_coordinates = [shape[20], shape[21], shape[40]]
            left_eye_leb_coordinates = [shape[22],shape[23],shape[42]] 

            leftEyeLEB = level_of_eyebrows(left_eye_leb_coordinates)
            rightEyeLEB = level_of_eyebrows(right_eye_leb_coordinates)

            leb = (rightEyeLEB + leftEyeLEB) / 2

            #size of pupil
            rightEyeSOP = size_of_pupil(leftEye)
            leftEyeSOP = size_of_pupil(rightEye)

            sop = (leftEyeSOP + rightEyeSOP) / 2

            # compute the convex hull for the left and right eye, then
            # visualize each of the eyes
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

            # draw mouth convex hull
            mouthHull = cv2.convexHull(shape[60:68])
            cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
            
            # draw eyebrow
            leftEyeBrow = shape[22:27]
            rightEyeBrow = shape[17:22]
            cv2.polylines(frame,[leftEyeBrow],False,(0,255,255),1)
            cv2.polylines(frame,[rightEyeBrow],False,(0,255,255),1)
            
            # check to see if the eye aspect ratio is below the blink
            # threshold, and if so, increment the blink frame counter
            if ear < ear_th:
                closeness=1
                COUNTER += 1
                perclos_list.append(1)
            # otherwise, the eye aspect ratio is not below the blink
            # threshold
            else:
                # if the eyes were closed for a sufficient number of
                # then increment the total number of blinks
                perclos_list.append(0)
                if COUNTER >= consec_th:
                    TOTAL += 1
                    blink_start = current_frame - COUNTER
                    blink_end = current_frame - 1

                # reset the eye frame counter
                COUNTER = 0
                closeness = 0

            # draw the total number of blinks on the frame along with
            # the computed eye aspect ratio for the frame
            cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "EAR: {:.2f}".format(ear), (340, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "PERCLOS: {:.2f}".format(perclos), (140, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "MAR: {:.2f}".format(mar), (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "MOE: {:.2f}".format(moe), (10, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "EC: {:.2f}".format(eye_circ), (10, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "LEB: {:.2f}".format(leb), (10, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "SOP: {:.2f}".format(sop), (10, 180),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "TIME: {:.1f}".format(current_frame/fps), (10, 280),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            # build frame_info dictionary then add to list
            frame_info = {
                'subject': subject,
                'external_factors': external_factors,
                'facial_actions': facial_actions,
                'frame_no': current_frame,
                'face_detected': 1,
                'left_ear': leftEAR,
                'right_ear': rightEAR,
                'avg_ear': ear,
                'mar': mar,
                'moe': moe,
                'left_ec' : leftEC,
                'right_ec' : rightEC,
                'avg_ec' : eye_circ,
                'left_leb': leftEyeLEB,
                'right_leb': rightEyeLEB,
                'avg_leb': leb,
                'left_sop': leftEyeSOP,
                'right_sop': rightEyeSOP,
                'avg_sop': sop,
                'closeness': closeness,
                'blink_no': TOTAL,
                'blink_start_frame': blink_start,
                'blink_end_frame': blink_end,
                'reserved_for_calibration': False,
                'perclos': -1,
            }

            if current_frame > skip_n:
                perclos = compute_perclos(perclos_list)
                perclos_list.pop(0)
                frame_info['perclos'] = perclos
    
        if current_frame <= skip_n:
            frame_info['reserved_for_calibration'] = True
        frame_info_list.append(frame_info)

        # append processed frame to list
        current_frame += 1
        if up_to == current_frame:
            break

    
    video_info_list.append(video_info_dict)

    frame_info_df = pd.DataFrame(frame_info_list)  # build a dataframe from frame_info_list
    video_info_df = pd.DataFrame(video_info_list)
    
    # print status
    file_name = os.path.basename(input_file)
    output_str = "Processing {} has done.\n\n".format(file_name)
    print(output_str)


    return frame_info_df, video_info_df

This is the main function where the file are called from the directory for processing the video and finally the result `frame_info_df` which have all frame related information and  `video_info_df` where all video related information is stored. This data is then converted into pickel file so that it can be used for the futher processing. `frame_info_df` and `video_info_df` will be stored as `{Subject}_{sub_folder}_{clean_name}_frame_info_df.pkl` and `{Subject}_{sub_folder}_{clean_name}_video_info_df.pkl` respectively.

During this stage all the video data is converted into usefull numerical data.


This process is performed for both training and testing data.

In [ ]:
#subjects=['035', '032', '023', '034', '036', '024', '001', '020', '031', '033', '015', '012', '009', '008', '005', '013', '006', '002']

directory = "/content/drive/My Drive/Project/Data/Training_Evaluation_Dataset/Training Dataset"
subjects=os.listdir(directory)
for subject in subjects:
    sub_folders = os.listdir(directory+'/'+subject)
    for sub_folder in sub_folders:
        if sub_folder == 'sunglasses':
            continue
        video_names=os.listdir(directory+'/'+subject+'/'+sub_folder)
        for video_name in video_names:
            clean_name = os.path.splitext(video_name)[0]
            extension = os.path.splitext(video_name)[1]
            if extension=='.avi': 
                file_path = directory+'/'+subject+'/'+sub_folder+'/'+video_name
                print(file_path)
                frame_info_df, video_info_dict = process_video(file_path, subject=subject, external_factors=sub_folder,facial_actions=clean_name, \
                                                                ear_th=EAR_THRESHOLD, consec_th=EAR_CONSEC_FRAMES, skip_n=SKIP_FIRST_FRAMES)
                
                train_path = "/content/drive/My Drive/Project/Data/train/"              

                frame_info_df.to_pickle('/content/drive/My Drive/Project/train/{}_{}_{}_frame_info_df.pkl'.format(subject,sub_folder,clean_name))
                video_info_dict.to_pickle('/content/drive/My Drive/Project/train/{}_{}_{}_video_info_df.pkl'.format(subject,sub_folder,clean_name))




/content/drive/My Drive/Project/Data/Training_Evaluation_Dataset/Training Dataset/035/glasses/yawning.avi
Processing yawning.avi has done.


/content/drive/My Drive/Project/Data/Training_Evaluation_Dataset/Training Dataset/035/glasses/slowBlinkWithNodding.avi
Processing slowBlinkWithNodding.avi has done.


/content/drive/My Drive/Project/Data/Training_Evaluation_Dataset/Training Dataset/035/glasses/nonsleepyCombination.avi
Processing nonsleepyCombination.avi has done.


/content/drive/My Drive/Project/Data/Training_Evaluation_Dataset/Training Dataset/035/glasses/sleepyCombination.avi
Processing sleepyCombination.avi has done.


/content/drive/My Drive/Project/Data/Training_Evaluation_Dataset/Training Dataset/035/nightglasses/yawning.avi
Processing yawning.avi has done.


/content/drive/My Drive/Project/Data/Training_Evaluation_Dataset/Training Dataset/035/nightglasses/slowBlinkWithNodding.avi
Processing slowBlinkWithNodding.avi has done.


/content/drive/My Drive/Project/Data/Training_

In [ ]:
directory = "/content/drive/My Drive/Project/Data/Training_Evaluation_Dataset/Evaluation Dataset"
subjects=os.listdir(directory)
for subject in subjects:
    video_names=os.listdir(directory+'/'+subject)
    for video_name in video_names:
        clean_name = os.path.splitext(video_name)[0]
        extension = os.path.splitext(video_name)[1]
        if extension=='.mp4': 
            external_factors =clean_name.split('_')[1]
            facial_actions=clean_name.split('_')[2]
            if external_factors == 'sunglasses':
                continue
            if external_factors == 'nightnoglasses':
                external_factors = 'night_noglasses'
            file_path = directory+'/'+subject+'/'+video_name
            print(file_path)
            frame_info_df, video_info_dict = process_video(file_path, subject=subject, external_factors=external_factors,facial_actions=facial_actions, \
                                                            ear_th=EAR_THRESHOLD, consec_th=EAR_CONSEC_FRAMES, skip_n=SKIP_FIRST_FRAMES)
            
            test_path = "/content/drive/MyDrive/Project/Data/test/"
            frame_plk_file = ('{}_{}_{}_frame_info_df.pkl').format(subject,external_factors,facial_actions)
            frame_plk_out = open(test_path + frame_plk_file,'wb')
            pickle.dump(frame_info_df,frame_plk_out)
            frame_plk_out.close()

            video_plk_file = ('{}_{}_{}_video_info_df.pkl').format(subject,external_factors,facial_actions)
            video_plk_out = open(test_path + video_plk_file,'wb')
            pickle.dump(video_info_dict,video_plk_out)
            video_plk_out.close()

            